In [1]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

In [3]:
device

'cuda:0'

In [4]:
model_id = "openai/whisper-large-v3" # This works very well, although some mistakes
model_id = "nvidia/parakeet-tdt-1.1b"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)

config.json: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1.27k/1.27k [00:00<00:00, 51.0kB/s]
model.safetensors: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3.09G/3.09G [00:46<00:00, 66.5MB/s]
generation_config.json: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 3.87k/3.87k [00:00<00:00, 174kB/s]


In [5]:
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

preprocessor_config.json: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 340/340 [00:00<00:00, 158kB/s]
tokenizer_config.json: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 283k/283k [00:00<00:00, 3.95MB/s]
vocab.json: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1.04M/1.04M [00:00<00:00, 8.81MB/s]
tokenizer.json: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2.48M/2.48M [00:00<00:00, 21.0MB/s]
merges.txt: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 494k/494k [00:00<00:00, 8.27MB/s]
normalizer.json: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 52.7k/52.7

In [ ]:
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)